In [ ]:
import os
import json
import warnings
import pandas as pd
import openai
from openai import OpenAI
import re
import requests
from bs4 import BeautifulSoup
from pathlib import Path
import re
from urllib.parse import urljoin, urlparse
import time
import argparse
from datetime import datetime,date
from io import StringIO
import csv
import country_converter as coco
import numpy as np

In [ ]:
# Please put your OpenAI API key here in place of the red string
client = OpenAI(api_key='')

In [ ]:
print("Upload the files here.\nPlease do not make any edits to the file(s) you downloaded from Sourcescrub.")
uploaded = files.upload()  # opens a file picker in the browser

Upload the files here.
Please do not make any edits to the file(s) you downloaded from Sourcescrub.


Saving Sourcescrub - Random List 2025.08.24 190558252.csv to Sourcescrub - Random List 2025.08.24 190558252 (10).csv


In [ ]:
def find_header_row(inputpath):
  with open(inputpath, "r") as f:
      for i in range(10):
          row=f.readline()
          if(row.startswith("Company Name,")):
            # print(i)
            return i

In [ ]:
def scrape_website(url, max_pages=5, delay=1, verbose=False):
    """
    Scrape all text content from a website.

    Args:
        url (str): The URL of the website to scrape.
        max_pages (int, optional): Maximum number of pages to scrape. Defaults to 10.
        delay (int, optional): Delay between requests in seconds. Defaults to 1.
        verbose (bool, optional): Whether to print progress information. Defaults to False.

    Returns:
        str: All text content from the website.
    """
    # Validate URL
    if not url.startswith(('http://', 'https://')):
        url = 'https://' + url

    # Initialize variables
    visited_urls = set()
    to_visit = [url]
    base_domain = urlparse(url).netloc
    all_text = []

    # Set up headers to mimic a browser
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
        'Accept-Language': 'en-US,en;q=0.5',
    }

    # Crawl the website
    page_count = 0
    while to_visit and page_count < max_pages:
        current_url = to_visit.pop(0)

        # Skip if already visited
        if current_url in visited_urls:
            continue

        # Add to visited set
        visited_urls.add(current_url)

        if verbose:
            print(f"Scraping: {current_url}")

        try:
            # Make the request
            response = requests.get(current_url, headers=headers, timeout=10)

            # Check if the request was successful
            if response.status_code != 200:
                if verbose:
                    print(f"Failed to retrieve {current_url}: Status code {response.status_code}")
                continue

            # Parse the HTML content
            soup = BeautifulSoup(response.text, 'html.parser')

            # Extract all text
            page_text = extract_text_from_soup(soup)
            all_text.append(f"--- Content from: {current_url} ---\n{page_text}\n")

            # Find all links on the page
            for link in soup.find_all('a', href=True):
                href = link['href']

                # Skip empty links, anchors, and non-HTTP links
                if not href or href.startswith('#') or href.startswith('javascript:') or href.startswith('mailto:'):
                    continue

                # Convert relative URLs to absolute URLs
                absolute_url = urljoin(current_url, href)

                # Only follow links to the same domain
                if urlparse(absolute_url).netloc == base_domain and absolute_url not in visited_urls:
                    to_visit.append(absolute_url)

            # Increment page count
            page_count += 1

            # Respect robots.txt by adding a delay
            time.sleep(delay)

        except Exception as e:
            if verbose:
                print(f"Error scraping {current_url}: {str(e)}")

    if verbose:
        print(f"Scraped {page_count} pages from {url}")

    # Join all text with newlines
    return '\n'.join(all_text)

def extract_text_from_soup(soup):
    """
    Extract all text from a BeautifulSoup object.

    Args:
        soup (BeautifulSoup): The BeautifulSoup object.

    Returns:
        str: All text content.
    """
    # Remove script and style elements
    for script_or_style in soup(['script', 'style', 'meta', 'noscript']):
        script_or_style.decompose()

    # Get text
    text = soup.get_text(separator=' ', strip=True)

    # Clean up text
    text = re.sub(r'\s+', ' ', text)  # Replace multiple whitespace with single space
    text = re.sub(r'\n\s*\n', '\n\n', text)  # Replace multiple newlines with double newlines

    return text.strip()

In [ ]:
def _get_encoding(model_name="gpt-4o"):
    try:
        import tiktoken
        try:
            return tiktoken.encoding_for_model(model_name)
        except Exception:
            # generic fallback works for GPT-4/3.5/4o families
            return tiktoken.get_encoding("cl100k_base")
    except Exception:
        return None  # no tiktoken installed

def count_tokens(text: str, model_name="gpt-4o") -> int:
    enc = _get_encoding(model_name)
    if enc:
        return len(enc.encode(text))
    # crude fallback: ~4 chars ≈ 1 token
    return max(1, len(text) // 4)

def truncate_to_tokens(text: str, max_tokens: int, model_name="gpt-4o-mini", reserve_tokens: int = 0) -> str:
    """Return text truncated to <= max_tokens, keeping a safety reserve."""
    effective = max(0, max_tokens - max(0, reserve_tokens))
    if effective <= 0:
        return ""
    enc = _get_encoding(model_name)
    if enc:
        ids = enc.encode(text)
        if len(ids) <= effective:
            return text
        return enc.decode(ids[:effective])
    # fallback by characters if no tiktoken
    max_chars = effective * 4
    return text[:max_chars]

In [ ]:
def extract_json(text):
    # Use regex to find the first {...} block
    match = re.search(r'\{.*?\}', text, re.DOTALL)
    if match:
        try:
            return json.loads(match.group())
        except json.JSONDecodeError as e:
            print("JSON decode error:", e)
    else:
        print("No JSON object found.")

In [ ]:
def generate_notes(company_name, company_description, employee_count, url):
    """
    Sends conversation messages to OpenAI and optionally includes
    attachment content in the first user prompt.
    """
    # Add attachment content to the first user message if exists
    website_data = scrape_website(url)
    capped_text = truncate_to_tokens(website_data, 20000, model_name="gpt-4o", reserve_tokens=1500)

    prompt_with_attachment = [
        {"role": "system", "content": "You are a helpful assistant that replies only in strict JSON, with keys: 'Name', 'Cheap Labor', 'Recurring', 'Category', 'Score', 'Notes'."},
        {"role": "user", "content": f"""I have to score (from 0-100) and take notes on this company {company_name} with this description:\n
        \"{company_description}\" with an employee count of {employee_count},\n and website data\n
        {capped_text}
        \n
        You are to first judge two things:\n

       1. If the company primarily operates as a low-cost outsourcing provider — relying heavily on labor from countries such as India, Pakistan, Croatia, the Philippines, Vietnam, Ukraine, Romania, or other regions where Western companies often outsource due to lower wage costs\n
        Analyze the company’s website and identify any indicators that it outsources or offshores labor. Specifically, look for: Mentions of offshore, nearshore, global delivery centers, or shared service centers. Lists of office locations in outsourcing hubs (e.g., India, Pakistan, Philippines, Vietnam, Romania, Ukraine, Croatia). Services described as Business Process Outsourcing (BPO), Knowledge Process Outsourcing (KPO), IT outsourcing, or call center operations. Careers or job postings concentrated in lower-cost regions. Language emphasizing cost savings, scalability, or providing “affordable” or “efficient” labor solutions. Case studies or client examples that highlight labor arbitrage.
        If so, set the ‘Cheap Labor’ field in the JSON to the boolean value ‘True’ and automatically assign a score of 0. \n

       2. The likelihood that the company follows a recurring revenue business model. To determine this, you need to analyze text data from the company’s website. Look for evidence of subscription or membership pricing (monthly, annual, or per-user), SaaS or cloud-based platforms with renewals, or service offerings provided on an ongoing basis such as managed services, retainers, or long-term contracts. Pay attention to language that references renewals, auto-renewal, continuous support, or recurring billing. Also check for mentions of Annual or Monthly Recurring Revenue in investor or press materials, customer success or account management functions tied to retention, and client examples that highlight continuous or long-term engagement. The description provided ealier may also help you figure this out.\n
        Using this information, assign a value of ‘Low,’ ‘Medium,’ or ‘High’ to indicate the likelihood that the company has a recurring revenue model, and record this in the ‘Recurring’ field of the JSON. If the likelihood is ‘Low,’ adjust your overall rating downward significantly, ensuring that the maximum cap is set at 50 instead of 100.\n

       Now, after taking note of these things, you are to do three things:\n

        First, assign the company to a category of your choice. Example categories include Construction and Design Software, Mass Tort Software, Cybersecurity, Healthcare, Fleet Management Software, Insurance, Managed Service Provider, and Food Tech, though you are free to create your own as appropriate. Once you have determined the category, record it in the ‘Category’ field of the JSON.\n

        Second, assign the company a score from 0 to 100, focusing primarily on the quality and sustainability of its business model, while also weighing industry attractiveness, market size, and potential for growth. Use the following standards:\n

        1. 75–100 (High Quality / Strong Candidate): The company demonstrates a clearly differentiated, defensible, and scalable business model (e.g., recurring revenue, high retention, strong unit economics, attractive margins). It operates in a large and growing market with favorable industry tailwinds, competitive advantages (technology, brand, or distribution), and evidence of execution capability. These companies are highly attractive and should be prioritized for investment consideration.\n
        2. 50–74 (Moderate Quality / Potential Candidate): The company has a sound but less compelling business model. Revenue may be partially recurring or dependent on cyclical demand, margins may be average, or customer retention may be moderate. The market opportunity exists but may be smaller, more competitive, or slower-growing. These companies may be worth considering depending on valuation, strategic fit, or further diligence.\n
        3. 25–49 (Low Quality / Weak Candidate): The company’s business model raises concerns (e.g., transactional or project-based revenue, high churn risk, reliance on low-cost labor arbitrage, commoditized offerings). Industry or market outlook may be limited, shrinking, or highly fragmented, with little competitive differentiation. These companies are unlikely to be strong candidates and typically should not move forward.\n
        4. 0–24 (Very Poor Quality / Exclude): The company has a fundamentally flawed or unsustainable business model (e.g., no clear path to profitability, highly concentrated revenue, declining market, no defensible moat). It operates in an unattractive or shrinking industry with severe headwinds. These companies should be removed from the sheet and excluded from consideration.\n

       The companies here should be judged with absolute rigor. Do not inflate scores—apply a cold, objective lens. In particular, avoid giving high marks to categories outside our focus: hardware manufacturers, law firms (note: legal services and legal software are acceptable), investment firms (though service providers for them may be relevant), hospitals, or traditional retail stores without a meaningful technology component. Our target profile is enterprise software and tech-enabled services businesses. These are the models that should earn the strongest scores.\n
       And when you are finished, put the numerical score under the 'Score' section on the json.\n

        Finally, add a note justifying the score, referencing specific aspects of the company, product, or market when relevant. Keep it under 40 words, in notes form (ideally not full sentences), and record it in the ‘Notes’ field of the JSON. Do not simply write phrases like ‘Strong recurring revenue’ or ‘Highly recurring revenue.’ Instead, explain why it is recurring, pointing to specific features of the product, customer contracts, or market dynamics.\n

        PLEASE respond in JSON format with keys: 'Name', 'Cheap Labor', 'Recurring', 'Category', 'Score', 'Notes'. I want your output to be ready to be converted to JSON \n\n"""}
    ]

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=prompt_with_attachment
    )
    assistant_reply = response.choices[0].message.content
    try:
        result = extract_json(assistant_reply)
        df = pd.DataFrame([result])
    except (json.JSONDecodeError, KeyError, ValueError, AttributeError):
        print("GPT Output Invalid; Attempting Prompt Again")
        df = generate_notes(company_name, company_description, employee_count, url)

    try:
        test_name = df.loc[0,'Name']
        test_cheap = df.loc[0,'Cheap Labor']
        test_recurring = df.loc[0,'Recurring']
        test_category = df.loc[0,'Category']
        test_score = df.loc[0,'Score']
        test_notes = df.loc[0,'Notes']
    except (KeyError, ValueError, AttributeError):
        print("GPT Output DataFrame Invalid:")
        print(df)
        print("Attempting Prompt Again")
        df = generate_notes(company_name, company_description, employee_count, url)

    return df

In [ ]:
def find_overlap(investors_str):
    if pd.isna(investors_str):
        return None
    check_list = [
    "ABRY Partners", "Accel Management", "Sequoia Capital",
    "Coatue Management", "Advent International", "Andreessen Horowitz",
    "Ares Management", "Audax Private Equity", "Bain Capital",
    "Battery Ventures", "Berkshire Partners", "Blackstone Group",
    "Clayton, Dubilier and Rice", "Clearlake Capital Group", "CVC Capital Partners",
    "Diversis Capital", "EQT", "Francisco Partners",
    "FTV Capital", "General Catalyst", "Genstar Capital",
    "GI Partners", "Golden Gate", "Great Hill Partners",
    "GTCR", "H.I.G. Capital", "HgCapital",
    "Index Ventures", "Institutional Venture Partners (IVP)", "Insight Partners",
    "JMI Equity", "Jerusalem Venture Partners (JVP)", "K1 Investment Management",
    "Kohlberg Kravis Roberts & Co.", "Kleiner Perkins", "Level Equity Management",
    "Lightspeed", "LLR Partners", "Main Capital Partners",
    "Marlin Equity Partners", "New Enterprise Associates", "New Mountain Capital",
    "Norwest Venture Partners", "OpenView Venture Partners", "Parthenon Capital Partners",
    "Permira", "Platinum Equity", "Providence Equity Partners",
    "Quad-C Management", "Riverside Partners", "Sageview Capital",
    "Serent Capital", "Silver Lake", "Spectrum Equity",
    "Stone Point Capital", " Summit Partners", "Susquehanna Growth Equity",
    "TA Associates Management", "Tiger Global Management",
    "The Carlyle Group", "Thoma Bravo", "Vista Equity Partners",
    "TPG", "Warburg Pincus", "Volition Capital",
    "Welsh, Carson, Anderson and Stowe Management", "PSG Equity"
]
    overlap = ""
    for company in check_list:
      if investors_str.find(company) != -1:
        if overlap != "":
          overlap = overlap + ", " + company
        else:
          overlap = company

    if overlap == "":
      return None
    else:
      return overlap

In [ ]:
#investors_str = "Sequoia Capital Operations LLC, Coatue Management"
#investors = {inv.strip() for inv in investors_str.split(",")}
#print(investors)

In [ ]:
def is_bad_country(country):
    if(pd.notna(country)):
      cc = coco.CountryConverter()
      continent = cc.convert(names=country, to='continent')
      region = cc.convert(names=country, to='UNregion')
      if(country == "IL" or region == "Northern America" or continent == "Europe" or continent == "Oceania"):
          return False
    return True

In [ ]:
floor = 0
ceiling = 100

def eval_growth(growth):
    if pd.isna(growth):
      return -1
    return max(floor, min(ceiling, 25 + 5 * growth))

def eval_raised(total_raised):
    if pd.isna(total_raised):
      return -1
    raised_in_mil = float(total_raised/1_000_000)
    if(raised_in_mil <= 5):
        return min(ceiling, 100 - 5 * raised_in_mil)
    else:
        return max(floor, 75 - 25 * (raised_in_mil - 5) / 15)

def eval_date(date_of_investment):
    if pd.isna(date_of_investment):
        return -1
    try:
        delta_date = (date.today() - date_of_investment.date()).days / 365.25
    except (TypeError, AttributeError, ValueError):
        return -2
    if(delta_date >= 6):
        return max(floor, min(ceiling, 5 * delta_date + 45))
    elif(delta_date <=5):
        return max(floor, min(ceiling, 15 * delta_date - 10))
    else:
        return max(floor, min(ceiling, 10 * delta_date + 15))

In [ ]:
def prefilter(output_df,ownership, investors, country,employee_count):
    if(ownership == "Public"):
        output_df.loc[0,'GPT Rationale'] = f"Tier 4: Public Company"
    elif(ownership == "Private Sub"):
        output_df.loc[0,'GPT Rationale'] = f"Tier 4: Private Subsidiary"
    elif(ownership == "Public Sub"):
        output_df.loc[0,'GPT Rationale'] = f"Tier 4: Public Subsidiary"
    elif(find_overlap(investors)):
        overlap = find_overlap(investors)
        output_df.loc[0,'GPT Rationale'] = f"Tier 4: {overlap} on the Cap Table"
    elif(is_bad_country(country)):
        output_df.loc[0,'GPT Rationale'] = f"Tier 4: Outside Our Preferred Geographies"
    elif(employee_count > 1000):
        output_df.loc[0,'GPT Rationale'] = f"Tier 4: Over 1000 Employees"
    elif(employee_count < 30):
        output_df.loc[0,'GPT Rationale'] = f"Tier 4: Under 30 Employees"
    else:
        return None

    return output_df

In [ ]:
def evaluate_companies(inputpath,outputpath):
    if inputpath.lower().endswith(".csv"):
        # company_info = pd.read_csv(inputpath, skiprows=find_header_row(inputpath), parse_dates=["Date of Most Recent Investment"], date_format="%m/%d/%y")
        company_info = pd.read_csv(inputpath, skiprows=find_header_row(inputpath))
        with warnings.catch_warnings():
            warnings.simplefilter("ignore", category=UserWarning)
            if company_info["Date of Most Recent Investment"].dtype == "O":
                company_info["Date of Most Recent Investment"] = pd.to_datetime(
                    company_info["Date of Most Recent Investment"],
                    errors="coerce",
                )
    elif inputpath.lower().endswith(".xlsx"):
        company_info = pd.read_excel(inputpath, skiprows=find_header_row(inputpath), engine="openpyxl")
    else:
        raise ValueError("Unsupported file type. Please upload a .csv or .xlsx file.")


    init = {
        "Name": [],
        "Tier": [],
        "Score": [],
        "GPT Rationale": [],
        "Category": [],
        "Recurring Revenue Likelihood": [],
        "Employee Count": [],
        "12 Months Growth Rate %": [],
        "Total Raised": [],
        "Investors": [],
        "Date of Most Recent Investment": [],
        "State": [],
        "Country": [],
        "Website": [],
        "LinkedIn": [],
    }
    all_output = pd.DataFrame([init])

    for index,row in company_info.iterrows():

        company_name = row['Company Name']

        print(f"\n{index+1}\n{company_name}\n")
        # if index == 100:
        #   break

        ownership = row['Ownership']
        investors = row['Investors']
        state = row['State']
        country = row['Country']

        growth = row['12 Months Growth Rate %']
        total_raised = row['Total Raised']
        date_of_investment = row['Date of Most Recent Investment']

        company_description = row['Description']
        employee_count = row['Employee Count']
        url = row['Website']
        linkedin = row['LinkedIn Account']

        output_df = pd.DataFrame([{
                  "Name": company_name,
                  "Tier": 4,
                  "Score": 0,
                  "GPT Rationale": f"",
                  "Category": "—",
                  "Recurring Revenue Likelihood": "—",
                  "Employee Count": employee_count,
                  "12 Months Growth Rate %": growth,
                  "Total Raised": total_raised,
                  "Investors": investors,
                  "Date of Most Recent Investment": date_of_investment,
                  "State": state,
                  "Country": country,
                  "Website": url,
                  "LinkedIn": linkedin
              }])


        mini = company_info.iloc[[index]].copy()
        cols = ['Company Name', 'Ownership', 'Investors','Country','12 Months Growth Rate %', 'Total Raised', 'Date of Most Recent Investment','Description','Employee Count','Website']
        blank_cells = mini.reindex(columns=cols).isna().sum(axis=1).iat[0]

        if blank_cells >= 8:
            output_df.loc[0,'GPT Rationale'] = f"Tier 4: incomplete information"
            print("Incomplete info")

        else:
            prefiltered_df = prefilter(output_df,ownership,investors,country,employee_count)

            if prefiltered_df is None:

                growth_score = int(eval_growth(growth))
                raised_score = int(eval_raised(total_raised))
                date_score = int(eval_date(date_of_investment))

                results_df = generate_notes(company_name, company_description, employee_count, url)
                # print("\nGPT Output:")
                # print(results_df)
                if(results_df.loc[0,'Cheap Labor']):
                  output_df.loc[0,'GPT Rationale'] = f"Tier 4: Uses cheap labor from countries that offer them"
                  print("Prefiltered: " + output_df['GPT Rationale'][0])

                else:

                  gpt_score = float(results_df['Score'][0])
                  print("GPT Score:" + str(gpt_score) + "; Growth Score:" + str(growth_score) + "; Total Raised Score:" + str(raised_score) + "; Date of Recent Inv. Score:" + str(date_score))


                  final_score = 4 * gpt_score
                  weights = 4
                  if(date_score > -1):
                    final_score += 0.5 * date_score
                    weights += 0.5
                  if(growth_score > -1):
                    final_score += 1.25 * growth_score
                    weights += 1.25
                  if(raised_score > -1):
                    final_score += 2 * raised_score
                    weights += 2
                  final_score = int(final_score / weights)

                  tier = 4 - int(final_score / 25)

                  output_df.loc[0,'Tier'] = np.float64(tier)
                  output_df.loc[0,'Score'] = np.float64(final_score)
                  output_df.loc[0,'GPT Rationale'] = results_df.loc[0,'Notes']
                  output_df.loc[0,'Category'] = results_df.loc[0,'Category']
                  output_df.loc[0,'Recurring Revenue Likelihood'] = results_df.loc[0,'Recurring']

                  print("GPT Analysis incorporated:")
                  print(output_df.loc[0,'GPT Rationale'])

            else:
                output_df.loc[0] = prefiltered_df.loc[0]
                print("Prefiltered: " + output_df['GPT Rationale'][0])

        # print(output_df)

        # print(output_df.iloc[0].values)
        all_output.loc[index,:] = output_df.iloc[0]

        # Append or write with headers
        # print("\nSaving to csv\n")
        all_output.to_csv(outputpath, mode='w', index=False)

In [ ]:
for filename in uploaded.keys():
    print(f"\n{filename}")
    output_filename = f"/content/Tiered_{filename}"
    evaluate_companies(filename,output_filename)

output_excel = "/content/Companies_Tiered_All.xlsx"

with pd.ExcelWriter(output_excel, engine="xlsxwriter") as writer:
    for filename in uploaded.keys():
        df = pd.read_csv(f"/content/Tiered_{filename}")
        sheet_name = filename.rsplit(".", 1)[0][:31]  # Excel sheet names max 31 chars
        df.to_excel(writer, sheet_name=sheet_name, index=False)

print(f"✅ Saved all tiered sheets into {output_excel}")


Sourcescrub - Random List 2025.08.24 190558252 (10).csv

1
Timeless Medical Systems, Inc.

GPT Score:85.0; Growth Score:96; Total Raised Score:-1; Date of Recent Inv. Score:-1
GPT Analysis incorporated:
High recurring from software solutions in pediatric neonatology; trusted in hospitals; extensive customer retention and safety-improving technology; large market potential in neonatal nutrition management.

2
EQT AB

Prefiltered: Tier 4: Over 1000 Employees

3
Assaia International AG

GPT Score:85.0; Growth Score:100; Total Raised Score:-1; Date of Recent Inv. Score:15
GPT Analysis incorporated:
Subscription model; strategic partnerships; strong AI-based optimization; global airport clientele; operational efficiency improvement.

4
Harvey,

Prefiltered: Tier 4: Sequoia Capital, Coatue Management, Kleiner Perkins on the Cap Table
✅ Saved all tiered sheets into /content/Companies_Tiered_All.xlsx


In [ ]:
files.download(output_excel)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>